In [1]:
import osmnx as ox
import pandana
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
target = [
    'rogers park', 'west ridge',
    'uptown', 'lincoln square',
    'north center', 'lake view', 'lincoln park',
    'near north side', 'edison park',
    'norwood park', 'jefferson park',
    'forest glen', 'north park', 'albany park',
    'portage park', 'irving park', 'dunning',
    'montclare', 'belmont cragin', 'hermosa',
    'avondale', 'logan square', 'humboldt park',
    'west town', 'austin', 'west garfield park',
    'east garfield park', 'near west side', 
    'north lawndale', 'south lawndale', 'lower west side', 'loop',
    'near south side', 'armour square', 'douglas',
    'oakland', 'fuller park', 'grand boulevard',
    'kenwood', 'washington park', 'hyde park',
    'woodlawn', 'south shore', 'chatham',
    'avalon park', 'south chicago', 'burnside',
    'calumet heights', 'roseland', 'pullman',
    'south deering', 'east side', 'west pullman',
    'riverdale', 'hegewisch', 'garfield ridge',
    'archer heights', 'brighton park', 'mckinley park',
    'bridgeport', 'new city', 'west elsdon',
    'gage park', 'clearing', 'west lawn',
    'chicago lawn', 'west englewood', 'englewood',
    'greater grand crossing', 'ashburn',
    'auburn gresham', 'beverly', 'washington heights',
    'mount greenwood', 'morgan park', 'ohare', 'edgewater'
]

In [3]:
tags = {
    'amenity' : [
        'cafe',
        'bar',
        'restaurant',
        'pub'
    ],
    'shop' : [
        'bakery',
        'convenience',
        'supermarket',
        'mall',
        'shoes',
        'clothes',
        'department_store'
    ],
    'leisure' : [
        'fitness_centre',
        'amusement_arcade',
        'bowling_alley'
    ]
}

In [38]:
# max finder for each col
from collections import deque
def findMax(x):
    queue = []
    max = 0
    for i, num in enumerate(x):
        if num > max:
            max = num
    return max

def findMin(x):
    queue = []
    min = 0
    for i, num in enumerate(x):
        if num > min:
            max = min
    return min

In [4]:
for city in target:
    cityname = f"{city}, chicago, illinois"
    crs = 3035
    graph = ox.graph_from_place(cityname, network_type='walk')
    graph = ox.projection.project_graph(graph, to_crs=crs)
    poi = ox.geometries.geometries_from_place(cityname, tags=tags)
    # project the place of interest
    poi = poi.to_crs(epsg=crs)

    max_time = 15
    walk_speed = 4.755
    for u, v, data in graph.edges(data=True):
        data['speed_kph'] = walk_speed
    graph = ox.add_edge_travel_times(graph)

    
    nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
    edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]

    # Construct the pandana network model
    network = pandana.Network(
        node_x=nodes['x'],
        node_y=nodes['y'], 
        edge_from=edges['u'],
        edge_to=edges['v'],
        edge_weights=edges[['travel_time']]
    )
    centroids = poi.centroid
    """ 
    Calculate and specifiy a max travel distance
    for further implications onto a df we create
    """
    max_distance = max_time * 60 #mins
    network.set_pois(
        category='pois',
        maxdist = max_distance,
        maxitems = 10,
        x_col = centroids.x,
        y_col = centroids.y
    )

    distances = network.nearest_pois(
        distance=max_distance,
        category = 'pois',
        num_pois = 10
    )

    distances.astype(int).head()
    df = distances
    break
    

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_12065/3800637937.py:6: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2263
Setting CH edge vector of size 7068
Range graph removed 7138 edges of 14136
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [5]:
df

,1,2,3,4,5,6,7,8,9,10
osmid,,,,,,,,,,
102825447,52.799999,67.599998,76.500000,292.700012,292.700012,304.100006,353.700012,357.799988,365.500000,365.500000
7618316017,44.000000,71.400002,73.500000,288.299988,298.600006,314.899994,359.600006,363.700012,371.399994,371.399994
5828278090,64.199997,67.900002,76.800003,289.299988,291.600006,293.000000,350.299988,354.399994,362.100006,362.100006
8988122479,52.799999,60.200001,69.099998,285.299988,285.299988,303.000000,346.299988,350.399994,358.100006,358.100006
8988122478,45.400002,60.200001,69.099998,285.299988,285.299988,310.399994,346.299988,350.399994,358.100006,358.100006
...,...,...,...,...,...,...,...,...,...,...
11169006287,340.700012,354.500000,399.899994,457.700012,555.299988,592.099976,669.700012,669.700012,669.700012,730.000000
11169006288,360.700012,374.500000,419.899994,477.700012,575.299988,612.099976,689.700012,689.700012,689.700012,750.000000
11169006377,78.199997,92.800003,92.800003,148.100006,148.100006,199.600006,199.600006,208.000000,208.000000,208.000000


In [41]:
df_max = df.max()
df_min = df.min()

In [43]:
x = findMax(df_max)
y = findMin(df_min)
X_std = (df - df.min(axis=0)) / (df.max(axis=0)) - df.min(axis=0)
X_scaled = X_std * (x - y) + y
print(X_scaled)

                     1           2           3            4             5   \
osmid                                                                        
102825447     85.146034   93.041744   98.300967 -6140.343502 -13574.385083   
7618316017    70.955029   98.271905   94.446028 -6145.578164 -13568.308183   
5828278090   103.529833   93.454655   98.686465 -6144.388474 -13575.518071   
8988122479    85.146034   82.856704   88.792114 -6149.147232 -13582.006989   
8988122478    73.212692   82.856704   88.792114 -6149.147232 -13582.006989   
...                 ...         ...         ...          ...           ...   
11169006287  549.417711  487.918624  513.863481 -5944.044757 -13303.911311   
11169006288  581.669997  515.445768  539.563080 -5920.250970 -13283.311632   
11169006377  126.106433  127.725951  119.246145 -6312.372590 -13723.320774   
11169121514  252.051620  279.262865  260.722427 -6247.177628 -13657.813802   
11459247156   51.603658   60.146810  154.197596 -6345.802868 -13